In [151]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, ShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, LassoCV, RidgeClassifier, Ridge
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipe
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipe

from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, \
classification_report, accuracy_score, precision_score, plot_confusion_matrix

### Loading the data

In [2]:
ls

Final_notebook.ipynb    README.md               Working_notebook.ipynb


In [3]:
df = pd.read_csv('/Users/raylinsoriano/Documents/Flatiron/capstone/data/listings (1).csv')

In [4]:
df

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2595,https://www.airbnb.com/rooms/2595,20210901220703,2021-09-02,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...",Centrally located in the heart of Manhattan ju...,https://a0.muscache.com/pictures/f0813a11-40b2...,2845,https://www.airbnb.com/users/show/2845,...,4.79,4.86,4.41,NaN,f,3,3,0,0,0.33
1,3831,https://www.airbnb.com/rooms/3831,20210901220703,2021-09-02,"Whole flr w/private bdrm, bath & kitchen(pls r...","Enjoy 500 s.f. top floor in 1899 brownstone, w...",Just the right mix of urban center and local n...,https://a0.muscache.com/pictures/e49999c2-9fd5...,4869,https://www.airbnb.com/users/show/4869,...,4.81,4.72,4.65,NaN,f,1,1,0,0,5.03
2,5121,https://www.airbnb.com/rooms/5121,20210901220703,2021-09-02,BlissArtsSpace!,<b>The space</b><br />HELLO EVERYONE AND THANK...,NaN,https://a0.muscache.com/pictures/2090980c-b68e...,7356,https://www.airbnb.com/users/show/7356,...,4.91,4.47,4.52,NaN,f,1,0,1,0,0.54
3,5136,https://www.airbnb.com/rooms/5136,20210901220703,2021-09-02,"Spacious Brooklyn Duplex, Patio + Garden",We welcome you to stay in our lovely 2 br dupl...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,7378,https://www.airbnb.com/users/show/7378,...,5.00,4.50,5.00,NaN,f,1,1,0,0,0.02
4,5178,https://www.airbnb.com/rooms/5178,20210901220703,2021-09-02,Large Furnished Room Near B'way,Please don’t expect the luxury here just a bas...,"Theater district, many restaurants around here.",https://a0.muscache.com/pictures/12065/f070997...,8967,https://www.airbnb.com/users/show/8967,...,4.40,4.86,4.36,NaN,f,1,0,1,0,3.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36918,52002317,https://www.airbnb.com/rooms/52002317,20210901220703,2021-09-02,HUGE ROOM IN IDEAL LES SPOT,Extremely Huge Room in the IDEAL Lower East Si...,NaN,https://a0.muscache.com/pictures/40bc2a16-7609...,283057520,https://www.airbnb.com/users/show/283057520,...,NaN,NaN,NaN,NaN,t,4,0,4,0,NaN
36919,52004427,https://www.airbnb.com/rooms/52004427,20210901220703,2021-09-02,Furnished Cozy Bedroom in East Village Location,"Full room in a 4 bedroom, 2 bathroom shared ho...",NaN,https://a0.muscache.com/pictures/361bd18b-3f23...,305240193,https://www.airbnb.com/users/show/305240193,...,NaN,NaN,NaN,NaN,f,307,56,251,0,NaN
36920,52005507,https://www.airbnb.com/rooms/52005507,20210901220703,2021-09-03,Bedroom Near Train w/ Washer Dryer,Bedroom Near Train w/ Washer Dryer,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,9372363,https://www.airbnb.com/users/show/9372363,...,NaN,NaN,NaN,NaN,f,3,0,3,0,NaN
36921,52005691,https://www.airbnb.com/rooms/52005691,20210901220703,2021-09-02,Furnished Cozy Bedroom in East Village Location,"Full room in a 3 bedroom, 1 bathroom shared ho...",NaN,https://a0.muscache.com/pictures/958369fa-05c1...,305240193,https://www.airbnb.com/users/show/305240193,...,NaN,NaN,NaN,NaN,f,307,56,251,0,NaN


In [5]:
#df_reviews = pd.read_csv('/Users/raylinsoriano/Documents/Flatiron/capstone/data/reviews (1).csv')

In [6]:
#df_reviews

In [7]:
df.shape

(36923, 74)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36923 entries, 0 to 36922
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            36923 non-null  int64  
 1   listing_url                                   36923 non-null  object 
 2   scrape_id                                     36923 non-null  int64  
 3   last_scraped                                  36923 non-null  object 
 4   name                                          36910 non-null  object 
 5   description                                   35710 non-null  object 
 6   neighborhood_overview                         22510 non-null  object 
 7   picture_url                                   36923 non-null  object 
 8   host_id                                       36923 non-null  int64  
 9   host_url                                      36923 non-null 

In [9]:
#pd.options.display.max_rows = 4000

In [10]:
df.duplicated().sum()

0

### Dropping empty columns and columns not needed

host_response_time                              
host_response_rate                              
host_acceptance_rate                           

#### Removing columns not needed

In [11]:
# df = df.drop(['host_url','listing_url', 'picture_url','host_thumbnail_url', 'host_picture_url','neighbourhood'
#              ,'property_type','bathrooms', 'minimum_minimum_nights', 'maximum_minimum_nights'
#              ,'minimum_maximum_nights','maximum_maximum_nights', 'minimum_nights_avg_ntm','maximum_nights_avg_ntm'
#              ,'calendar_updated','calendar_last_scraped', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
#              'first_review','last_review', 'review_scores_value','license','reviews_per_month', 'neighborhood_overview'
#              ,'host_response_rate', 'host_acceptance_rate','host_about', 'host_location', 'scrape_id','last_scraped'], axis =1)

In [12]:
df.shape

(36923, 74)

In [13]:
df.isna().sum()

id                                                 0
listing_url                                        0
scrape_id                                          0
last_scraped                                       0
name                                              13
                                                ... 
calculated_host_listings_count                     0
calculated_host_listings_count_entire_homes        0
calculated_host_listings_count_private_rooms       0
calculated_host_listings_count_shared_rooms        0
reviews_per_month                               9296
Length: 74, dtype: int64

In [14]:
median = df.filter(['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin',
        'review_scores_communication', 'review_scores_location']).median()

In [15]:
cols = ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin',
        'review_scores_communication', 'review_scores_location']                          
df[cols]=df[cols].fillna(median)

In [16]:
#df.dropna(inplace=True)

In [17]:
df.shape

(36923, 74)

In [18]:
#df.dropna(inplace=True)

In [19]:
#df['review_scores_rating'].hist();

In [20]:
#new_rating = df[(df['review_scores_rating'] >= 4.0)]

In [21]:
# fig, ax = plt.subplots(figsize= (10,10))
# ax.hist(x = new_rating['review_scores_rating'], bins = 100)
# plt.show()

In [22]:
#new_rating['review_scores_rating'].value_counts()

In [23]:
#df['review_scores_rating'].describe()

In [24]:
#df = df.reset_index()

In [25]:
#df.head()

In [26]:
df.shape

(36923, 74)

#### Creating new columns

In [27]:
# Created a column for how many years the airbnb has been a host
df['host_since'] = pd.to_datetime(df['host_since'])
df['year'] = df['host_since'].apply(lambda date: date.year)

df['years_been_host'] = 2021 - df['year']

In [28]:
#Created a column of how many verifications each host has
df['number_of_verifications'] = df.host_verifications.str.count(',')+1

In [29]:
#Created a column of how many amenities each host has
df['number_of_amenities'] = df.amenities.str.count(',')+1

In [30]:
#df['amenities'].str.split(",", n = 49, expand = True)

In [31]:
df['neighbourhood_cleansed']

0                   Midtown
1        Bedford-Stuyvesant
2        Bedford-Stuyvesant
3               Sunset Park
4                   Midtown
                ...        
36918       Lower East Side
36919          East Village
36920               Midwood
36921          East Village
36922        Hell's Kitchen
Name: neighbourhood_cleansed, Length: 36923, dtype: object

#### removing rows not needed

In [32]:
df = df.drop(['host_url','listing_url', 'picture_url','host_thumbnail_url', 'host_picture_url','neighbourhood', 'host_neighbourhood'
             ,'property_type','bathrooms', 'minimum_minimum_nights', 'maximum_minimum_nights'
             ,'minimum_maximum_nights','maximum_maximum_nights', 'minimum_nights_avg_ntm','maximum_nights_avg_ntm'
             ,'calendar_updated','calendar_last_scraped', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
             'first_review','last_review', 'review_scores_value','license','reviews_per_month', 'neighborhood_overview'
             ,'host_response_rate', 'host_acceptance_rate','host_about', 'host_location', 'scrape_id','last_scraped'
             ,'host_since','year', 'neighbourhood_cleansed', 'amenities','host_name', 'name'], axis =1)



In [33]:
df.isna().sum()

id                                                  0
description                                      1213
host_id                                             0
host_response_time                              15743
host_is_superhost                                 111
host_listings_count                               111
host_total_listings_count                         111
host_verifications                                  0
host_has_profile_pic                              111
host_identity_verified                            111
neighbourhood_group_cleansed                        0
latitude                                            0
longitude                                           0
room_type                                           0
accommodates                                        0
bathrooms_text                                    105
bedrooms                                         3936
beds                                              611
price                       

In [34]:
df.dropna(inplace=True)

In [35]:
df.shape

(17841, 41)

In [36]:
# import nums_from_string
# numbers = []
# for word in df.bathrooms_text.str.split():
#     if word is isdigit():
#         numbers.append(int(word))

# print(numbers)
#numbers = [int(word) for word in a_string.split() if word.isdigit()]

#df['bt'] = df['bathrooms_text'].apply(nums_from_string.get_nums(df['bathrooms_text'].to_string()))
#df['bt'] = df['bathrooms_text'].apply([i for i in nums_from_string.get_nums(df['bathrooms_text'].to_string())])

# for a in nums_from_string.get_nums(df['bathrooms_text'].to_string()):
#                                    numbers.append(a)
#type(df.bathrooms_text.to_string)
#nums_from_string.get_nums(df['bathrooms_text'].to_string())

#### Imbalanced Target

In [38]:
# df['price'] = df['price'].str.replace('$', '').str.replace(',', '')
# df['price'] = df.price.astype(float)

In [39]:
df['has_availability'] = df['has_availability'].map({'t': 0,
                                     'f': 1,})

## Modeling

### Train/Test Split

In [82]:
X= df.drop(["has_availability",'host_id','id', 'latitude','longitude'],axis=1)
y= df["has_availability"]

X_train, X_test, y_train, y_test= train_test_split(X, y, stratify=y)

#### Creating a Cleaning pipline

In [83]:
#Categorical variables to use
categorical_vars = ["host_response_time","host_has_profile_pic","host_identity_verified","host_is_superhost"
            ,"neighbourhood_group_cleansed","room_type","instant_bookable", "bathrooms_text"]

# Numerical Variables to use
# num_vars = ['price','minimum_nights','bedrooms', 'beds', "reviews_per_month","host_listings_count",
#            "calculated_host_listings_count_shared_rooms", "host_response_rate", "host_acceptance_rate",
#            "review_scores_cleanliness","review_scores_value","review_scores_rating", "number_of_reviews"]

In [84]:
train = X_train[categorical_vars]

# Create OneHotEncoder object to create dummies
ohe = OneHotEncoder(handle_unknown='ignore')
enc = ohe.fit(train)
train_dummies_trans = enc.transform(train)

# Dummies values in matrix form
train_data = train_dummies_trans.todense()

# New dummy column names
names = ohe.get_feature_names(categorical_vars)

# Make them into Dataframe
train_dummies_trans_df = pd.DataFrame(train_data, columns=names,index = X_train.index)

In [85]:
# test_dummies = X_test[categorical_vars]
# Create OneHotEncoder object to create dummies
# ohe = OneHotEncoder(handle_unknown='ignore')

# # Transform the dataset into dummies matrix
# enc = ohe.fit(test_dummies)
# test_dummies_trans = enc.transform(test_dummies)

# # Dummies values in matrix form
# test_data = test_dummies_trans.todense()

# # New dummy column names
# names = ohe.get_feature_names(categorical_vars)

# # Make them into Dataframe
# test_dummies_trans_df = pd.DataFrame(test_data, columns=names,index = X_test.index)

In [86]:
numerical = X_train.select_dtypes(exclude='object') 
numerical = pd.DataFrame(numerical, index = X_train.index)
df_train = train_dummies_trans_df.merge(numerical, left_index = True , right_index=True)

In [100]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13380 entries, 31602 to 24948
Data columns (total 73 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   host_response_time_a few days or more         13380 non-null  float64
 1   host_response_time_within a day               13380 non-null  float64
 2   host_response_time_within a few hours         13380 non-null  float64
 3   host_response_time_within an hour             13380 non-null  float64
 4   host_has_profile_pic_f                        13380 non-null  float64
 5   host_has_profile_pic_t                        13380 non-null  float64
 6   host_identity_verified_f                      13380 non-null  float64
 7   host_identity_verified_t                      13380 non-null  float64
 8   host_is_superhost_f                           13380 non-null  float64
 9   host_is_superhost_t                           13380 non-n

In [88]:
df_train.head()

,host_response_time_a few days or more,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,host_has_profile_pic_f,host_has_profile_pic_t,host_identity_verified_f,host_identity_verified_t,host_is_superhost_f,host_is_superhost_t,...,review_scores_checkin,review_scores_communication,review_scores_location,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,years_been_host,number_of_verifications,number_of_amenities
31602,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,4.96,4.98,4.88,1,1,0,0,8.0,5,13
9191,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,4.88,4.96,4.67,5,4,1,0,7.0,6,32
30071,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,4.96,4.98,4.88,18,3,15,0,1.0,5,15
27612,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,3.00,2.00,2.00,128,7,121,0,7.0,7,9
867,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,4.94,4.93,4.86,2,2,0,0,9.0,5,24


In [90]:
# df_train_new = cleaning_pipeline.fit_transform(df_train)
# df_test_new = cleaning_pipeline.transform(df_test)

In [91]:
# # Continuous features
# continuous = df_train.select_dtypes(exclude='object') 

# #train_continuous = df_train[continuous]
# #test_continuous = X_test[continuous]

# # Make StandardScaler object
# scaler = StandardScaler()

# # Fit the train data into the scaler
# df_fit = scaler.fit_transform(df_train)

# # Transform data
# #train_data_scaled = df_fit.transform(train_continuous)
# #test_data_scaled = df_fit.transform(test_continuous)
# #names = scaler.get_feature_names(df_fit)
# # Make them into Dataframe
# train_scaled_df = pd.DataFrame(df_fit)
# #test_scaled_df = pd.DataFrame(test_data_scaled, columns = continuous)


#### Creating a function that prints out validation score

In [149]:
def print_cv_scores(pipe, X, y):
    '''
    Runs cross_validate on given feature and class target arrays using given pipeline, 
    printing the scoring results for both training and cross_val.
    '''
    scoring = ['accuracy','precision', 'recall', 'f1']
    
    results = cross_validate(pipe, X, 
                                   y, 
                                   return_train_score=True, scoring=scoring, cv=2)
    
    print(results['train_accuracy'])
    print('Training Accuracy', results['train_accuracy'].mean())
    #print('##############')
    #print(results['test_accuracy'])
    #print('Cross_Val Accuracy', results['test_accuracy'].mean())
    print('##############')
    print('Training Precision:', results['train_precision'].mean())
    #print('Cross_Val Precision:', results['test_precision_macro'].mean())
    print('##############')
    print('Training  Recall:', results['train_recall'].mean())
    #print('Cross_Val Recall:', results['test_recall_macro'].mean())
    print('##############')
    print('Training  F1:', results['train_f1'].mean())
    #print('Cross_Val  F1:', results['test_f1_macro'].mean())

### First Simple Model (baseline)

In [94]:
# from joblib._parallel_backends import LokyBackend

# class NestedBackend(LokyBackend):
#     def get_nested_backend(self):
#         backend = NestedBackend()
#         backend.nested_level = 0
#         return backend, None

# from joblib import Parallel, delayed
# def print_pid(level=0):
#     import os
#     import time
#     time.sleep(.2)
#     print(f"Level {level}: {os.getpid()}")


# def nested():
#     print_pid(1)
#     Parallel(n_jobs=2)(delayed(print_pid)(2) for _ in range(2))


# backend = NestedBackend()
    
# print_pid() 
# Parallel(n_jobs=2, backend=backend)(delayed(nested)() for _ in range(2))


In [136]:
lasso_pip = Pipeline([('lo', Lasso(max_iter=100000))
                    ])

In [137]:
lasso_pip.fit(df_train, y_train)

Pipeline(steps=[('lo', Lasso(max_iter=100000))])

In [150]:
print_cv_scores(lasso_pip, df_train, y_train)

[nan nan]
Training Accuracy nan
##############
Training Macro Precision: nan
##############
Training Macro Recall: nan
##############
Training Macro F1: nan


/Users/raylinsoriano/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/raylinsoriano/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/raylinsoriano/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 87, in __call__
    score = scorer._score(cached_call, estimator,
  File "/Users/raylinsoriano/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 242, in _score
    return self._sign * self._score_func(y_true, y_pred,
  File "/Users/raylinsoriano/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "/Users/raylinsoriano/anaconda3/lib/python3.8/site-packages/sklearn/metric

In [68]:
fsm_pipe = Pipeline([('lg', LogisticRegression(max_iter=100000, verbose=2, n_jobs=-1))
                    ])



In [65]:
 fsm_pipe.fit(df_train,  y_train)

Level 2: 89372
Level 2: 89374
Level 2: 89375
Level 2: 89373


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           80     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  9.27431D+03    |proj g|=  3.96812D+06

At iterate    1    f=  7.68360D+03    |proj g|=  2.42564D+04

At iterate    2    f=  5.00217D+02    |proj g|=  2.02873D+04

At iterate    3    f=  3.21743D+02    |proj g|=  1.52035D+04

At iterate    4    f=  2.64475D+02    |proj g|=  8.08143D+03

At iterate    5    f=  1.65930D+02    |proj g|=  4.29948D+03

At iterate    6    f=  1.65269D+02    |proj g|=  6.95211D+02

At iterate    7    f=  1.64122D+02    |proj g|=  3.19162D+03

At iterate    8    f=  1.60664D+02    |proj g|=  9.59636D+03

At iterate    9    f=  1.59321D+02    |proj g|=  3.60253D+03

At iterate   10    f=  1.58908D+02    |proj g|=  3.49721D+02

At iterate   11    f=  1.58862D+02    |proj g|=  3.19116D+02

At iterate   12    f=  1.58831D+02    |proj g|=  4.85209D+02

At iterate   13    f=  1.5


At iterate  141    f=  1.08640D+02    |proj g|=  1.28095D+03

At iterate  142    f=  1.08564D+02    |proj g|=  6.20224D+02

At iterate  143    f=  1.08395D+02    |proj g|=  1.41734D+02

At iterate  144    f=  1.08326D+02    |proj g|=  1.28987D+02

At iterate  145    f=  1.08311D+02    |proj g|=  4.62586D+02

At iterate  146    f=  1.08274D+02    |proj g|=  2.17738D+02

At iterate  147    f=  1.08169D+02    |proj g|=  2.24193D+02

At iterate  148    f=  1.08022D+02    |proj g|=  6.03344D+02

At iterate  149    f=  1.07841D+02    |proj g|=  6.44184D+02

At iterate  150    f=  1.07821D+02    |proj g|=  4.50891D+02

At iterate  151    f=  1.07751D+02    |proj g|=  6.08999D+01

At iterate  152    f=  1.07747D+02    |proj g|=  2.31583D+01

At iterate  153    f=  1.07746D+02    |proj g|=  5.24372D+01

At iterate  154    f=  1.07744D+02    |proj g|=  1.47483D+01

At iterate  155    f=  1.07743D+02    |proj g|=  2.99830D+01

At iterate  156    f=  1.07740D+02    |proj g|=  5.17635D+01

At iter


At iterate  275    f=  1.05286D+02    |proj g|=  6.54796D+02

At iterate  276    f=  1.05233D+02    |proj g|=  1.70175D+02

At iterate  277    f=  1.05206D+02    |proj g|=  8.58113D+01

At iterate  278    f=  1.05189D+02    |proj g|=  4.30242D+01

At iterate  279    f=  1.05179D+02    |proj g|=  4.77162D+01

At iterate  280    f=  1.05146D+02    |proj g|=  7.94534D+01

At iterate  281    f=  1.05086D+02    |proj g|=  2.22552D+02

At iterate  282    f=  1.05076D+02    |proj g|=  1.83089D+02

At iterate  283    f=  1.05028D+02    |proj g|=  4.14626D+01

At iterate  284    f=  1.05010D+02    |proj g|=  5.98425D+01

At iterate  285    f=  1.05004D+02    |proj g|=  1.69430D+02

At iterate  286    f=  1.04987D+02    |proj g|=  1.02789D+02

At iterate  287    f=  1.04979D+02    |proj g|=  2.93497D+01

At iterate  288    f=  1.04976D+02    |proj g|=  1.94377D+02

At iterate  289    f=  1.04969D+02    |proj g|=  1.97066D+01

At iterate  290    f=  1.04963D+02    |proj g|=  9.44198D+01

At iter

...


At iterate 1401    f=  1.02093D+02    |proj g|=  3.61044D+01

At iterate 1402    f=  1.02092D+02    |proj g|=  9.03701D+00

At iterate 1403    f=  1.02092D+02    |proj g|=  8.55721D+00

At iterate 1404    f=  1.02092D+02    |proj g|=  4.80724D+00

At iterate 1405    f=  1.02092D+02    |proj g|=  1.22930D+01

At iterate 1406    f=  1.02092D+02    |proj g|=  4.79375D+00

At iterate 1407    f=  1.02092D+02    |proj g|=  5.94116D+00

At iterate 1408    f=  1.02092D+02    |proj g|=  1.50940D+01

At iterate 1409    f=  1.02092D+02    |proj g|=  2.33123D+01

At iterate 1410    f=  1.02092D+02    |proj g|=  2.16687D+01

At iterate 1411    f=  1.02092D+02    |proj g|=  2.71375D+01

At iterate 1412    f=  1.02092D+02    |proj g|=  1.41768D+01

At iterate 1413    f=  1.02092D+02    |proj g|=  3.19387D+00

At iterate 1414    f=  1.02092D+02    |proj g|=  8.42537D-01

At iterate 1415    f=  1.02092D+02    |proj g|=  1.49221D+00

At iterate 1416    f=  1.02092D+02    |proj g|=  3.42255D+00

At iter


At iterate 1536    f=  1.02083D+02    |proj g|=  2.26320D+01

At iterate 1537    f=  1.02083D+02    |proj g|=  2.71311D+01

At iterate 1538    f=  1.02083D+02    |proj g|=  1.38901D+01

At iterate 1539    f=  1.02082D+02    |proj g|=  1.88756D+00

At iterate 1540    f=  1.02082D+02    |proj g|=  9.11129D+00

At iterate 1541    f=  1.02082D+02    |proj g|=  1.61442D+01

At iterate 1542    f=  1.02082D+02    |proj g|=  1.30632D+01

At iterate 1543    f=  1.02082D+02    |proj g|=  4.27088D+01

At iterate 1544    f=  1.02082D+02    |proj g|=  3.31852D+01

At iterate 1545    f=  1.02081D+02    |proj g|=  1.46872D+01

At iterate 1546    f=  1.02081D+02    |proj g|=  4.98413D+00

At iterate 1547    f=  1.02081D+02    |proj g|=  7.84549D+00

At iterate 1548    f=  1.02081D+02    |proj g|=  4.53857D+00

At iterate 1549    f=  1.02081D+02    |proj g|=  2.26367D+01

At iterate 1550    f=  1.02081D+02    |proj g|=  8.40660D+00

At iterate 1551    f=  1.02081D+02    |proj g|=  6.09866D+00

At iter


At iterate 1674    f=  1.02079D+02    |proj g|=  7.12664D-01

At iterate 1675    f=  1.02079D+02    |proj g|=  1.98035D+00

At iterate 1676    f=  1.02079D+02    |proj g|=  7.83910D-01

At iterate 1677    f=  1.02079D+02    |proj g|=  2.85803D+00

At iterate 1678    f=  1.02079D+02    |proj g|=  5.38765D+00

At iterate 1679    f=  1.02079D+02    |proj g|=  1.05438D+00

At iterate 1680    f=  1.02079D+02    |proj g|=  1.07072D+00

At iterate 1681    f=  1.02079D+02    |proj g|=  6.02042D-01

At iterate 1682    f=  1.02079D+02    |proj g|=  3.51245D+00

At iterate 1683    f=  1.02079D+02    |proj g|=  1.65617D+00

At iterate 1684    f=  1.02079D+02    |proj g|=  1.28254D+00

At iterate 1685    f=  1.02079D+02    |proj g|=  5.99308D+00

At iterate 1686    f=  1.02079D+02    |proj g|=  3.47337D+00

At iterate 1687    f=  1.02079D+02    |proj g|=  1.57089D+00

At iterate 1688    f=  1.02079D+02    |proj g|=  1.46678D+00

At iterate 1689    f=  1.02079D+02    |proj g|=  2.78160D+00

At iter

[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   27.7s finished


Pipeline(steps=[('lg',
                 LogisticRegression(max_iter=100000, n_jobs=-1, verbose=2))])

In [ ]:
print_cv_scores(fsm_pipe, df_train, y_train)

In [ ]:
#print_cv_linear(fsm_pipe, train_scaled_df, y_train)

In [ ]:
# # Linear Regression (check MAE, MSE, RMSE)


# from sklearn.linear_model import LinearRegression
# #splitter = ShuffleSplit(n_splits=3, test_size=0.25, random_state=0)
# model = LinearRegression()

# model_scores = cross_validate(
#     estimator=model,
#     X=train_scaled_df,
#     y=y_train,
#     return_train_score=True,
    
# )

# print("Train score:  ", model_scores["train_score"].mean())

In [ ]:
# model.fit(train_scaled_df, y_train)

# preds_train = model.predict(train_scaled_df)

# print(f'Train MAE: {mean_absolute_error(y_train, preds_train)}')
# print(f'Train MSE: {mean_squared_error(y_train, preds_train)}')
# print(f'Train RMSE: {mean_squared_error(y_train, preds_train, squared=False)}')
# print(f'Train R-Squared: {r2_score(y_train, preds_train)}')


In [139]:
#model.intercept_

In [ ]:
# # Check coefficients for each features
# coef = dict(zip(train_scaled_df.columns, model.coef_))

# sorted(coef.items(), key=lambda x:x[1], reverse=True)

In [ ]:
# numeric_features = X_train.select_dtypes(exclude='object')
# numeric_transformer = Pipeline(steps=[
#     ('scaler', StandardScaler())])

# categorical_features = ["host_response_time","host_has_profile_pic","host_identity_verified","host_is_superhost"
#             ,"neighbourhood_group_cleansed","room_type","instant_bookable", "bathrooms_text", "has_availability"]
# categorical_transformer = Pipeline(steps=[
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)])

# clf = Pipeline(steps=[('preprocessor', preprocessor),
#                        ('regressor',  LinearRegression())])

# clf['preprocessor'][1][1]['onehot'].get_feature_names(categorical_features)